In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
import wave
import copy
import math
import tensorflow as tf
from keras.models import Sequential, Model, load_model
from keras.layers.core import Dense, Activation
# from keras.layers import LSTM, Input, Flatten, Merge, Bidirectional
from keras.layers import LSTM, Input, Flatten, Bidirectional
from sklearn.preprocessing import label_binarize

from features import *
from helper import *
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from tkinter import TclError

# use this backend to display in separate Tk window
%matplotlib tk

# constants

TIME_SEC = 1
RATE = 44100                 # samples per second
CHUNK = 44100 * TIME_SEC #1024 * 20 #int(44100 / TIME_SEC)             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone


p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

Using TensorFlow backend.


{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 4, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Speakers (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 

In [2]:
# create matplotlib figure and axes
#fig, ax = plt.subplots(1, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

import sounddevice as sd
print(sd.query_devices())

print("starting stream declaration")

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK,
    input_device_index = 1
)

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Microphone (Realtek(R) Audio), MME (2 in, 0 out)
   2 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  3 Speakers (Realtek(R) Audio), MME (0 in, 2 out)
   4 BenQ GW2480 (Intel(R) Display A, MME (0 in, 2 out)
   5 Primary Sound Capture Driver, Windows DirectSound (2 in, 0 out)
   6 Microphone (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
   7 Primary Sound Driver, Windows DirectSound (0 in, 2 out)
   8 Speakers (Realtek(R) Audio), Windows DirectSound (0 in, 2 out)
   9 BenQ GW2480 (Intel(R) Display Audio), Windows DirectSound (0 in, 2 out)
  10 BenQ GW2480 (Intel(R) Display Audio), Windows WASAPI (0 in, 2 out)
  11 Speakers (Realtek(R) Audio), Windows WASAPI (0 in, 2 out)
  12 Microphone (Realtek(R) Audio), Windows WASAPI (2 in, 0 out)
  13 Speakers (Realtek HD Audio output), Windows WDM-KS (0 in, 8 out)
  14 Microphone 1 (Realtek HD Audio Mic input with SST), Windows WDM-KS (2 in, 0 out)
  15 Microphone 2 (Realtek

In [3]:
# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)
fig , ax = plt.subplots()
# create a line object with random data
#line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)

# basic formatting for the axes
ax.set_title('Emotion Prediction')
ax.set_xlabel('Time')
ax.set_ylabel('Confidence')
# ax.set_ylim(0, 255)
# ax.set_xlim(0, 2 * CHUNK)
# plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

ax.plot([0], label="Anger", color='r')
ax.plot([0], label="Excited", color='y')
ax.plot([0], label="Neutral", color='g')
ax.plot([0], label="Sad", color='b')
plt.legend(loc="upper left")

# show the plot
plt.show(block=False)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

stream started


In [4]:
# set up model
model = load_model("C:/Uni Shit/EECS 4088/IEMOCAP-Emotion-Detection/data/Trained_models/2_layer_LSTM.pickle")
cols = ['ang', 'exc', 'neu', 'sad']
df_pred_wav = pd.DataFrame(columns=cols)
data_prev = []
graph_window = 15

In [5]:
print("starting loop")
while (True):
    
    # binary data
    data_new = stream.read(CHUNK)

    #convert data to integers, make np array, then offset it by 127
    data_new = struct.unpack(str(2 * CHUNK) + 'B', data_new)

    data_int = np.append(data_prev, data_new)

    # Generate features from data
    st_features = calculate_features(data_int, RATE, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)

    # reshape input from (34, 100) to (1, 100, 34)
    st_features = np.array([st_features.T])

    # predict on model
    wav_test_results = model.predict(st_features)

    df_pred_wav = df_pred_wav.append({cols[0]:wav_test_results[0][0],
                        cols[1]:wav_test_results[0][1],
                        cols[2]:wav_test_results[0][2],
                        cols[3]:wav_test_results[0][3]
    }, ignore_index=True)

    df_pred_wav_view = df_pred_wav[df_pred_wav.shape[0] - graph_window : ]

    #plt.cla()

    ax.axhline(y=0.5, color='r', linestyle='--')
    ax.plot(df_pred_wav_view['ang'], label="Anger", color='r')
    ax.plot(df_pred_wav_view['exc'], label="Excited", color='y')
    ax.plot(df_pred_wav_view['neu'], label="Neutral", color='g')
    ax.plot(df_pred_wav_view['sad'], label="Sad", color='b')

    # # create np array and offset by 128
#     data_np = np.array(data_int, dtype='b')[::2] + 128
    
#     line.set_ydata(data_np)
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))

    data_prev = data_new


starting loop


KeyboardInterrupt: 

In [ ]:
# stop stream (6)
stream.stop_stream()
stream.close()
wf.close()

# close PyAudio (7)
p.terminate()

In [ ]:
# define callback (2)
def callback(in_data, frame_count, time_info, status):
    data = wf.readframes(frame_count)
    # data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    with open(LOG_PATH + "log_file.txt", "a") as f:
            f.write(str(data) + "\n")

    return (data, pyaudio.paContinue)

In [ ]:
# set up model
model = load_model("C:/Uni Shit/EECS 4088/IEMOCAP-Emotion-Detection/data/Trained_models/2_layer_LSTM.pickle")
cols = ['ang', 'exc', 'neu', 'sad']
df_pred_wav = pd.DataFrame(columns=cols)
data_prev = []
graph_window = 15

In [ ]:
print("starting loop")
while (True):
    
    # binary data
    data_new = stream.read(CHUNK)

    #convert data to integers, make np array, then offset it by 127
    data_new = struct.unpack(str(2 * CHUNK) + 'B', data_new)

    #calculate the threshold

    data_int = np.append(data_prev, data_new)

    # Generate features from data
    st_features = calculate_features(data_int, RATE, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)

    # reshape input from (34, 100) to (1, 100, 34)
    st_features = np.array([st_features.T])

    # predict on model
    wav_test_results = model.predict(st_features)

    df_pred_wav = df_pred_wav.append({cols[0]:wav_test_results[0][0],
                        cols[1]:wav_test_results[0][1],
                        cols[2]:wav_test_results[0][2],
                        cols[3]:wav_test_results[0][3]
    }, ignore_index=True)

    df_pred_wav_view = df_pred_wav[df_pred_wav.shape[0] - graph_window : ]

    #plt.cla()

    ax.axhline(y=0.5, color='r', linestyle='--')
    ax.plot(df_pred_wav_view['ang'], label="Anger", color='r')
    ax.plot(df_pred_wav_view['exc'], label="Excited", color='y')
    ax.plot(df_pred_wav_view['neu'], label="Neutral", color='g')
    ax.plot(df_pred_wav_view['sad'], label="Sad", color='b')

    # # create np array and offset by 128
#     data_np = np.array(data_int, dtype='b')[::2] + 128
    
#     line.set_ydata(data_np)
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))

    data_prev = data_new


In [ ]:
# stop stream (6)
stream.stop_stream()
stream.close()
wf.close()

# close PyAudio (7)
p.terminate()

In [ ]:
# define callback (2)
def callback(in_data, frame_count, time_info, status):
    data = wf.readframes(frame_count)
    # data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    with open(LOG_PATH + "log_file.txt", "a") as f:
            f.write(str(data) + "\n")

    return (data, pyaudio.paContinue)

In [ ]:
wf = wave.open(WAV_PATH, 'rb')

# instantiate PyAudio (1)
p = pyaudio.PyAudio()

# open stream using callback (3)
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                input = True,
                stream_callback=callback)

In [ ]:
# start the stream (4)
stream.start_stream()


In [ ]:
# stop stream (6)
stream.stop_stream()
stream.close()
wf.close()

# close PyAudio (7)
p.terminate()